**Loading WSDREAM files**

* userlist.txt: contains the data pertaining to the users
* wslist.txt: contains the data pertaining to the services
* rtMatrix.txt: contains the data pertaining to the Response Time QoS observations
* tpMatrix.txt: contains the data pertaining to the Throuput QoS observations

In [1]:
import numpy as np
import pandas as pd

# Read the data and convert it into dataframes to facilitate exploration
usersData = pd.read_csv("userlist.txt", sep='\t', encoding='latin-1')
servicesData = pd.read_csv("wslist.txt", sep='\t', encoding='latin-1')
rtData = pd.read_csv("rtMatrix.txt", sep='\t', encoding='latin-1', header = None)
tpData = pd.read_csv("tpMatrix.txt", sep='\t', encoding='latin-1', header = None)

In [2]:
# Drop the last column from Rt and TP data as they don't have any information
rtData.drop(rtData.columns[-1], axis=1, inplace=True)
tpData.drop(tpData.columns[-1], axis=1, inplace=True)

In [3]:
# Calculate the fill value (the value to be used to fill missing data instead of -1 in the dataset)
rt_fillValue = rtData.median().mean()
tp_fillValue = tpData.median().mean()

# Implement the change
rtData.replace(-1,rt_fillValue, inplace=True)
tpData.replace(-1,tp_fillValue, inplace=True)

In [4]:
# Drop unnecessary features (columns) from the data describing the users and services
# We'll leave the following features Country, AS, Latitude and Longitude. Hence we'll drop "IP Address" and "IP_No." in both 
# datasets + "WSDL Address" and "Service Provider" from the service set.

usersData.drop(['IP Address', 'IP_No.'], axis=1, inplace=True)
servicesData.drop(['WSDL Address', 'Service Provider', 'IP Address','IP No.'], axis=1, inplace=True)

In [5]:
# Change the Country and AS fields to categorical data and then add category numbers as a field to the set

usersData[['Country','AS']] = usersData[['Country','AS']].astype('category')
servicesData[['Country','AS']] = servicesData[['Country','AS']].astype('category')

usersData['Country_catcod'] = usersData['Country'].cat.codes
usersData['AS_catcod'] = usersData['AS'].cat.codes
usersData.head()

servicesData['Country_catcod'] = servicesData['Country'].cat.codes
servicesData['AS_catcod'] = servicesData['AS'].cat.codes
servicesData.head()

,Service_ID,Country,AS,Latitude,Longitude,Country_catcod,AS_catcod
0,0,United States,AS3356 Level 3 Communications,38.0,-97.0,70,571
1,1,United States,AS3356 Level 3 Communications,38.0,-97.0,70,571
2,2,United States,AS3356 Level 3 Communications,38.0,-97.0,70,571
3,3,United States,AS3356 Level 3 Communications,38.0,-97.0,70,571
4,4,United States,AS3356 Level 3 Communications,38.0,-97.0,70,570


In [6]:
context_rtData = pd.concat([tpData, usersData[['Country_catcod','AS_catcod','Latitude','Longitude']]], axis=1, sort=False)
context_rtData.rename({'Country_catcod': 5825, 'AS_catcod': 5826, 'Latitude': 5827, 'Longitude': 5828}, axis=1, inplace=True)
context_rtData.head()

context_tpData = pd.concat([tpData, usersData[['Country_catcod','AS_catcod','Latitude','Longitude']]], axis=1, sort=False)
context_tpData.rename({'Country_catcod': 5825, 'AS_catcod': 5826, 'Latitude': 5827, 'Longitude': 5828}, axis=1, inplace=True)
context_tpData.head()

,0,1,2,3,4,5,6,7,8,9,...,5819,5820,5821,5822,5823,5824,5825,5826,5827,5828
0,0.334,17.543,25.316,9.049,9.009,5.692,4.415,189.045,5.181,4.622,...,1.381,0.740000,0.616000,0.762,0.596000,0.590000,29,115,38.0000,-97.0000
1,0.938,15.267,21.978,7.968,7.874,7.025,3.067,166.666,4.555,4.166,...,10.714,14.814000,13.937000,1.733,15.748000,15.209000,29,115,38.0464,-122.2300
2,2.341,10.928,15.957,5.602,5.586,26.086,3.081,109.183,3.115,2.473,...,4.918,30.260547,30.260547,0.382,30.260547,30.260547,15,94,35.6850,139.7514
3,2.886,17.699,25.751,9.090,9.132,8.720,2.614,134.930,5.390,5.434,...,16.949,23.952000,28.169000,18.433,23.529000,23.121000,29,28,40.4249,-86.9162
4,2.309,17.621,25.751,9.090,9.049,8.196,2.392,123.986,5.376,4.846,...,32.967,7.042000,43.478000,24.390,44.444000,42.105000,29,28,40.4249,-86.9162


In [7]:
# Write the resulting matrices to new csv files
context_rtData.to_csv(r'responseTimeWithContext.csv', float_format='%.3f', header=False, index=False)
context_tpData.to_csv(r'throughputWithContext.csv', float_format='%.3f', header=False, index=False)